# Package, module setting

In [1]:
import torch
import sys
import time
import yaml
import torchvision
from torchvision import transforms, datasets

import numpy as np
import os
from torch import cuda
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

In [2]:
sys.path.append('../')
from mymodels.resnet_base_network import ResNet18
from mydata.indexLoader import myDataUnit, nolabel_dataset

In [3]:
batch_size = 32
data_transforms = torchvision.transforms.Compose([transforms.ToTensor()])

config = yaml.load(open("../config/config.yaml", "r"), Loader=yaml.FullLoader)

# device = 'cpu'
device = 'cuda' if cuda.is_available() else 'cpu'
print(f"Training with: {device}")
if device=='cuda':
    torch.cuda.empty_cache()


Training with: cuda


# images dataloader
- shuffled one
- not shuffled one
- dataset
- dataloader

In [4]:
image_list = []
curr_psnr_sum = 30
# file_path = "./image_name_less" + str(curr_psnr_sum)+ ".txt"
# encoder_out_dir = str(curr_psnr_sum)+"db"

# file_path = "./image_names_all"+ ".txt"
# encoder_out_dir = "all"


file_path = "final_names.txt"
encoder_out_dir = "final"

f = open(file_path, 'r')
lines = f.readlines()
for line in lines:
    listed = line.split(" ")
    image_list.append(listed[0].strip())
f.close()


In [5]:
def name_to_int(imgname):
    lhs, rhs = imgname.split("_")
    strint = int(lhs)*1000 + int(rhs)
    return int(strint)

In [6]:
data_path = '/home/hong/dir1/final_eval/HR' # without last slash

path_n_name = []
for img_name in image_list:
    path_n_name.append( tuple((data_path+'/'+img_name+'.png', name_to_int(img_name))) )

# shuffle the pairs for training
# import random
# random.shuffle(path_n_name)

print('path_n_name example: ', path_n_name[0])
print(len(path_n_name))

path_n_name example:  ('/home/hong/dir1/final_eval/HR/10_10.png', 10010)
1000


In [7]:
my_dataset = nolabel_dataset(path_n_name, transform=data_transforms)

In [8]:
print("Input shape:", my_dataset[0][0].shape)

Input shape: torch.Size([3, 512, 512])


In [9]:
my_dataloader = DataLoader(my_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=False, pin_memory=True)

# Encoder loading
load encoder for both images

In [10]:
encoder = ResNet18(**config['network'])
output_feature_dim = encoder.projetion.net[0].in_features
print(output_feature_dim)

512


In [11]:
#load pre-trained parameters, BYOL pretrained model
load_params = torch.load(os.path.join('../runs/0118_run/checkpoints/model.pth'),
                        map_location=torch.device(torch.device(device)))

if 'online_network_state_dict' in load_params:
    encoder.load_state_dict(load_params['online_network_state_dict'])
    print("Parameters successfully loaded.")

# remove the projection head
encoder = torch.nn.Sequential(*list(encoder.children())[:-1])    
encoder = encoder.to(device)

Parameters successfully loaded.


# Encoding

In [12]:
def get_features_from_encoder(encoder, loader):
    
    x1_train = []
    y_train = []
    for x1, y in tqdm(loader):
        x1=x1.to(device)

        with torch.no_grad():
            f_vector1 = encoder(x1)
            f_vector1 = f_vector1.to('cpu')
            x1_train.extend(f_vector1)
            y_train.extend(y)

    x1_train = torch.stack(x1_train)
    y_train = torch.tensor(y_train)
    
    return x1_train, y_train

In [13]:
encoder.eval()
x1, y = get_features_from_encoder(encoder, my_dataloader)

if len(x1.shape) > 2:
    print(x1.shape)
    x1 = torch.mean(x1, dim=[2, 3])
    
print("Training data shape:", x1.shape)

100%|██████████| 32/32 [00:20<00:00,  1.58it/s]

torch.Size([1000, 512, 10, 10])
Training data shape: torch.Size([1000, 512])


In [14]:
# save tensor array
m1 = { 'x1': x1, 'y' : y}

date_dir = "./tensors/%s/byol_encoder_out/" % (encoder_out_dir)
if os.path.exists(date_dir) == False:
    os.mkdir(date_dir)
tensor_path = date_dir+"x2.pt"  ## os.path.join()
torch.save(m1, tensor_path)
